In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

dataset = pd.read_table('Data/data_banknote_authentication.txt', sep = ",", header=None)
X = dataset.iloc[:, :-1].values
t = dataset.iloc[:, -1].values

# split the training set, validation set and test set of ratio 6:2:2
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=8775)
X_train, X_valid, t_train, t_valid = train_test_split(X_train, t_train, test_size=0.25, random_state=8775)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_valid = sc.transform(X_valid)

In [17]:
clf = MLPClassifier( activation='relu', 
                    solver='sgd', 
                    learning_rate='constant', 
                    learning_rate_init=0.05,
                    hidden_layer_sizes=(5, 2), 
                    n_iter_no_change=100,
                    early_stopping=True)